# Предобработка данных и подготовка датасетов для ML - моделей

In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = ['account_id', 'name', 'point', 'call_count', 'total_call_time', 'total_exclusive_time', 'min_call_time', 'max_call_time', 'sum_of_squares', 'instances', 'language', 'app_name', 'app_id', 'scope', 'host', 'display_host', 'pid', 'agent_version', 'labels']
data = pd.read_csv('metrics_collector.tsv',
                   sep='\t',
                   header=0,
                   names=columns)
data.head()

C:\Users\Artur\AppData\Local\Temp\ipykernel_12156\3948894926.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('metrics_collector.tsv',


,account_id,name,point,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,instances,language,app_name,app_id,scope,host,display_host,pid,agent_version,labels
0,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:33:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
1,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:34:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
2,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
3,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:36:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
4,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:37:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}


# Анализ данных

## Работа с признаками

По умолчанию для работы нам даны 19 признаков:

* account_id;
* name;
* point;
* call_count;
* total_call_time;
* total_exclusive_time;
* min_call_time;
* max_call_time;
* sum_of_squares;
* instances;
* language;
* app_name;
* app_id;
* scope;
* host;
* display_host;
* pid;
* agent_version;
* labels,

однако среди них есть те, которые не представляют интереса для создания ML-модели, или предсказания выбросов, ввиду отсутствия различий в значениях для разных элементов столбца.

Для того чтобы получить информацию по уникальным значениям для каждого столбца, а также по количеству данных уникальных значений, воспользуемся методом .value_counts(), тем самым мы сможем понять, какие признаки являются неинформативными, что позволит понизить размерность данных, без потери качества.




In [3]:
# выведем дополнительную информацию о размерности данных
print('Размерность данных составляет {:,} строк и {:,} столбцов'.format(*list(data.shape)))

Размерность данных составляет 18,363,749 строк и 19 столбцов


Рассмотрим признак __account_id__ содержащий информацию об идентефикаторе аккаунта, с которого происходит выгрузка данных.

Для всех предоставленных данных - 18 363 749 значениям, в данной колонке соответствует единственное значение - ***1***. Следовательно можно отнести признак к неинформативным.

In [4]:
data['account_id'].value_counts()

account_id
1    18363749
Name: count, dtype: int64

Признак **language** с информацией о языке программирования также содержит единственное значение - ***java***, которое также не добавляет данным никакой информативности.

In [5]:
data['language'].value_counts()

language
java    18363749
Name: count, dtype: int64

Для признака **app_name** характерно наличие одного значения - **[GMonit] Collector**, которое используется для выбора данных для подсчета метрик. Однако в нашем случае, ввиду того, что данные являются неинформативными мы также избавляемся от данного признкака, как от ненужного.


In [6]:
data['app_name'].value_counts()

app_name
[GMonit] Collector    18363749
Name: count, dtype: int64

Столбец **app_id** аналогично не содержит полезной информации, т.к. имеет единственное значение - ***17592186045423***. Из чего следует, что его также нужно подвергунть удалению.


In [7]:
data['app_id'].value_counts()

app_id
17592186045423    18363749
Name: count, dtype: int64

Среди оставшихся признаков у нас имеется еще два неинформативных признака - **pid** и **labels**, содержащие также по 1 значению - ***1*** и ***{}*** соответственно. Следовательно нам нужно избавиться и от этих признаков тоже.

In [8]:
print(data['pid'].value_counts(), data['labels'].value_counts(), sep='\n\n')

pid
1    18363749
Name: count, dtype: int64

labels
{}    18363749
Name: count, dtype: int64


Преобразуем данные, удалив неинфомративные столбцы. Тем самым мы понизим размерность данных без потери информации

In [9]:
data = data.drop(['account_id', 'language', 'app_name', 'app_id', 'pid', 'labels'], axis=1)
data.head()

,name,point,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,instances,scope,host,display_host,agent_version
0,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:33:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:34:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
2,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
3,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:36:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
4,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:37:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0


Переведем дату и время в формат datetime, а также выведем основную информацию о датафрейме

In [10]:
# переведем время и дату в формат datetime
data['point'] = pd.to_datetime(data['point'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363749 entries, 0 to 18363748
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   name                  object        
 1   point                 datetime64[ns]
 2   call_count            float64       
 3   total_call_time       float64       
 4   total_exclusive_time  float64       
 5   min_call_time         float64       
 6   max_call_time         float64       
 7   sum_of_squares        float64       
 8   instances             float64       
 9   scope                 object        
 10  host                  object        
 11  display_host          object        
 12  agent_version         object        
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 1.8+ GB


## Рассчет метрик на данных

### Web Response

Рассчитаем метрику Web Response на наших данных, с учетом условий для их рассчета

> Web Response - время ответа сервиса на внешний http-запрос

Нам доступен код SQL-запроса:

```sql
select
 point as time,
 sumOrNull(total_call_time) / sumOrNull(call_count) as " "
from metrics_collector
where
 language = 'java'
 and app_name = '[GMonit] Collector'
 and scope = ''
 and name = 'HttpDispatcher'
group by time
order by time
```

После выполнения этого запроса нам необходимо получить время и дату, а также значение метрики в виде суммы значений переменной **total_call_time** разделенной на сумму значений переменной **call_count**


Также в запросе присутствует упорядочивание данных по дате и времени в порядке возрастания, помимо этого данные следует выбирать по следующим условиям:

```sql
where
 language = 'java'
 and app_name = '[GMonit] Collector'
 and scope = ''
 and name = 'HttpDispatcher'
```

После очистки данных мы автоматически учитываем условия переменных **language**, **app_name**. Осталось учесть данные для двух последних условий и получить результат

Создаем маски для первого набора данных и получаем интересующий результат

In [11]:
mask1 = data['scope'].isna()
mask2 = data['name'] == 'HttpDispatcher'
web_response = data[mask1 | mask2]

Импортируем библиотеку warnings для блокировки появления информации о предупреждениях, при работе с большими данными

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
web_response.head()

,name,point,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,instances,scope,host,display_host,agent_version
0,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:33:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:34:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
2,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
3,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:36:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0
4,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:37:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,575f5ba20b4b,575f5ba20b4b,8.5.0


Находим метрику для каждого конкретного значения в конкретный момент времени и заполняем все значения *Nan* нулями

In [14]:
web_response['web_response'] = web_response['total_call_time'] / web_response['call_count']
web_response['web_response'] = web_response['web_response'].fillna(0)

Для получения верной метрики группируем данные по дате и времени, а значения метрик в каждой из записей суммируем по датам

In [15]:
result = web_response.groupby('point')['web_response'].sum()

Округляем значения до 3 знаков после запятой (ввиду их больших значений высокая точность не требуется), после чего сохраняем первый файл с метрикой - web_response.csv

In [16]:
result = result.apply(lambda x: round(x, 3))

In [17]:
result.to_csv('web_response.csv', index=True)

### Throughput

Рассчитаем метрику throughput на найденных данных

> Throughput - пропускная способность сервиса. Измеряется в запросах в минуту.

Нам снова доступен код SQL-запроса:

```sql
select
 point as time,
 sumOrNull(call_count)
from metrics_collector
where
 language = 'java'
 and app_name = '[GMonit] Collector'
 and scope = ''
 and name = 'HttpDispatcher'
group by time
order by time
```

Как видно, то что касается условия выбора данных при **where** наши запросы для *web response* и *throughput* - идентичны. Следовательно воспользуемся уже готовыми данными и получим значение метрики.

Заполним *Nan* нулями в столбце *call_count*, т.к. именно от него зависит вторая метрика

In [18]:
web_response['call_count'] = web_response['call_count'].fillna(0)

Проведем аналогичные действия, что и для нахождения первой метрики. Группируем данные по дате и времени, суммируем значения,
округляем результаты до 3 знаков после запятой, сохраняем файл со второй метрикой.

In [19]:
result2 = web_response.groupby('point')['call_count'].sum()
result2 = result2.apply(lambda x: round(x, 3))
result2.to_csv('throughput.csv', index=True)

### Apdex

> APDEX - сводный синтетический показатель “здоровья” сервиса. Изменяется от 0 до 1. Чем ближе к 1, тем лучше.

```sql
with
 sumOrNull(call_count) as s,
 sumOrNull(total_call_time) as t,
 sumOrNull(total_exclusive_time) as f
select
 point as time,
 (s + t/2) / (s + t + f) as " "
from metrics_collector
where
 language = 'java'
 and app_name = '[GMonit] Collector'
 and scope = ''
 and name = 'Apdex'
group by time
order by time

```

В нашем случае SQL-запрос отличается от ранее приведенных только наличием нового условия:
```sql
 and name = 'Apdex'
```

а также измененным кодом для нахождения значения метрики

```sql
with
 sumOrNull(call_count) as s,
 sumOrNull(total_call_time) as t,
 sumOrNull(total_exclusive_time) as f
select
 point as time,
 (s + t/2) / (s + t + f) as " "
```

Пропишем новые маски для данных и выведем отдельный датасет для поиска метрики

In [20]:
mask3 = data['scope'].isna()
mask4 = data['name'] == 'Apdex'
data3 = data[mask3 | mask4]

Заполним значения во всех трех столбцах (избавимся от Nan), т.к. наличие пропусков способно ухудшить результаты оценки

In [21]:
data3['call_count'] = data3['call_count'].fillna(0)
data3['total_call_time'] = data3['total_call_time'].fillna(0)
data3['total_exclusive_time'] = data3['total_exclusive_time'].fillna(0)

Проведем математические вычисления для нахождения метрики

In [22]:
data3['apdex'] = (data3['call_count'] + data3['total_call_time']/2) / (data3['call_count'] + data3['total_call_time'] + data3['total_exclusive_time'])

Сгруппируем данные, найдем сумму значений по датам и сохраним файл с метрикой

In [23]:
result3 = data3.groupby('point')['apdex'].sum()
result3 = result3.apply(lambda x: round(x, 3))
result3.to_csv('apdex.csv', index=True)

### Error

> Error - процент ошибок в обработанных запросах
 
```sql
select
 point as time,
 sumOrNullIf(call_count, name='Errors/allWeb') / sumOrNullIf(call_count, name='HttpDispatcher') as " "
from metrics_collector
where
 language = 'java'
 and app_name = '[GMonit] Collector'
 and scope = ''
 and name in ('HttpDispatcher', 'Errors/allWeb')
group by time
order by time
```

В данном случае метрика зависит от нескольких условий, во-первых подсчет *call_count* ведется только если значение в столбце **name** равняется "Errors/allWeb", делится же результат на значения при том же столбце, однако со значением **name** равном "HttpDispatcher"

Также обратим внимание на новое условие where, которое требует двух значений в столбце name для нахождения метрики:

```sql
where
 language = 'java'
 and app_name = '[GMonit] Collector'
 and scope = ''
 and name in ('HttpDispatcher', 'Errors/allWeb')
```

Напишем новые маски, для получения нужной части данных

In [24]:
mask5 = data['scope'].isna()
mask6 = data['name'] == 'HttpDispatcher'
mask7 = data['name'] == 'Errors/allWeb'

Объединяем условия и получаем новые данные

In [26]:
data4 = data[(mask6 | mask7) & mask5]

Группируем и получаем отдельно значения для 'name' == 'Errors/allWeb' и 'name' == 'HttpDispatcher'

In [21]:
sub_result4 = data[data['name'] == 'HttpDispatcher'].groupby('point')['call_count'].sum()
sub_result5 = data[data['name'] == 'Errors/allWeb'].groupby('point')['call_count'].sum()

Делим одно значение на другое, как то указано в SQL-запросе

In [26]:
result4 = sub_result5 / sub_result4
result4 = result4.fillna(0)

Сохраняем полученные данные с агрегированной метрикой

In [28]:
result4.to_csv('error.csv', index=True)